In [ ]:
import os
import pandas as pd
import requests
import spacy
import string
from typing import List, Tuple, Union, Callable, Dict, Iterator
from collections import defaultdict
from difflib import SequenceMatcher
from spacy.matcher import Matcher, PhraseMatcher
from spacy.tokens import Doc, Token, Span
from spacy.matcher import Matcher
!python -m spacy download en_core_web_md

In [ ]:
matcher = Matcher(nlp.vocab)

# This pattern was created from the available cases under the redacted 
# directory in our shared HRF google drive

pattern = [
            # most specific phrases
            [{"LOWER": "court"}, {"LOWER": "finds"}, {"LOWER": "respondent"}, {"LOWER": "generally"}, {"LOWER": "credible"}],
            [{"LOWER": "court"}, {"LOWER": "finds"}, {"LOWER": "respondent"}, {"LOWER": "testimony"}, {"LOWER": "credible"}],
            [{"LOWER": "court"}, {"LOWER": "finds"}, {"LOWER": "respondent"}, {"LOWER": "credible"},

            # standard phrases
            [{"LOWER": "credible"}, {"LOWER": "witness"}],
            [{"LOWER": "generally"}, {"LOWER": "consistent"}],
            [{"LOWER": "internally"}, {"LOWER": "consistent"}],
            [{"LOWER": "sufficiently"}, {"LOWER": "consistent"}],
            [{"LOWER": "testified"}, {"LOWER": "credibly"}],
            [{"LOWER": "testimony"}, {"LOWER": "credible"}],
            [{"LOWER": "testimony"}, {"LOWER": "consistent"}],

            # least specific phrases
            [{"LOWER": "coherent"}],
            [{"LOWER": "plausible"}]
          ]

matcher.add('credibility', pattern)

In [ ]:
def similar(target_phrases, file):
    """GET RID OF PUNCT"""
    matcher = Matcher(nlp.vocab)
    matcher.add('target_phrases', target_phrases)
    
    return matcher(file, as_spans=True)

In [ ]:
def get_credibility(self):  
    """
    Returns whether or not the Respondent was identified as credible by their
    assigned judge / court. 
    """ 
    pattern = [
                # most important phrasing 
                [{"LOWER": "court"}, {"LOWER": "finds"}, {"LOWER": "respondent"}, {"LOWER": "generally"}, {"LOWER": "credible"}],
                [{"LOWER": "court"}, {"LOWER": "finds"}, {"LOWER": "respondent"}, {"LOWER": "testimony"}, {"LOWER": "credible"}],
                [{"LOWER": "court"}, {"LOWER": "finds"}, {"LOWER": "respondent"}, {"LOWER": "credible"},

                # next in omportance
                [{"LOWER": "credible"}, {"LOWER": "witness"}],
                [{"LOWER": "generally"}, {"LOWER": "consistent"}],
                [{"LOWER": "internally"}, {"LOWER": "consistent"}],
                [{"LOWER": "sufficiently"}, {"LOWER": "consistent"}],
                [{"LOWER": "testified"}, {"LOWER": "credibly"}],
                [{"LOWER": "testimony"}, {"LOWER": "credible"}],
                [{"LOWER": "testimony"}, {"LOWER": "consistent"}],

                # least importance 
                [{"LOWER": "coherent"}],
                [{"LOWER": "plausible"}]
              ]

    similar_cred = similar(target_phrases=pattern, file=self.doc)
    
    if similar_cred:
        # following code adds matches to a list, may or may not be needed in
        # the front end.
        
        # cred = []
        # for phrase in similar_cred:
        #     if phrase.text.lower() not in cred:
        #         cred.append(phrase.text.lower())

        return 'Respondent was found credible' # this could be changed to bool

    return 'Respondent was not found credible' # same for this

In [ ]:
# Non-credible wording:

# In some decisions, the immigration judge will find parts of the testimony 
# “problematic,” or question its plausibility, without actually reaching a 
# conclusion that the testimony lacked credibility. In such cases, argue on 
# appeal that the statutory presumption of credibility should apply.

# Exhibited behavior that does not always seem reasonable, nor plausible, 
# nor....worthy of belief because of inconsistent behavior and inconsistent 
# statements.

# Seemed rehearsed, lacked the ring of truth
# Highly implausible
# Couldn't produce
# Perceived lack of rationality, persuasiveness, and consistency
# Did not cry during testimony, appeared highly peeved
# Did not request asylum immediately upon entry to US



# Credible wording:

# Findings of fact and conclusion of law
# A. Credibility
# The court finds the respondent to have testified credibly

# Analysis
# B. Credibility and Corroboration
# The parties stipulated that the respondent filed a credible claim. The court finds that () testified credibly.

# Asylum
# A. Credibility and Corroboration
# the Court finds Respondent's testimony to be credible, provided a sufficiently detailed, plausible, and internally consistent accounts of the events

# Decisions and orders of the immigration judge
# III. Credibility and Corroboration
# the Court shall find that Respondent is generally credible.

# Legal Standards and Analysis
# A. Credibility
# Court finds that Respondent's testimony was credible. ...testimony was detailed, plausible, internally consistent, and consistent with documentary evidence.
# credible witnesses corroborated the Respondent's claim.

# Findings of fact and conclusion of law
# A. Credibility
# Court finds the Respondent to have testified credibly